Loading everything

In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import scipy.sparse as sp
import numpy as np
from codecs import open
import networkx as nx
import community
import random
from time import time
from settings import API_KEY
from infomap import infomap

ImportError: No module named 'infomap'

In [2]:
DATA_FOLDER = '/home/teven/fake_news/Wikidumps/'
RAW_FOLDER = os.path.join(DATA_FOLDER, 'raw/')
LINKS_FILE = os.path.join(DATA_FOLDER, '2110_symmetrized_links.json')
FINAL_INDEX_FILE = os.path.join(DATA_FOLDER, '2110_final_index.json')
ADJACENCY_MATRIX = os.path.join(DATA_FOLDER, '2110_conflict_adjacency.txt.npz')
MATRIX_INDEX = os.path.join(DATA_FOLDER, '2110_conflict_matrix_index.txt')
TALK_DATA_FILE = os.path.join(DATA_FOLDER, '2110_talk_data.json')
CONTROVERSY_SCORES = os.path.join(DATA_FOLDER, '2110_controversy_scores.txt')
REVERSE_INDEX_FILE = os.path.join(DATA_FOLDER, '2110_reverse_index.json')
EDIT_WARS = os.path.join(DATA_FOLDER, '2601_conflicts.json')

In [3]:
wars = json.load(open(EDIT_WARS))
wars = [item for item in wars if item is not None]
print ('done')
wars_links = json.load(open(LINKS_FILE))
print ('done')
wars_links = {war: wars_links.get(str(war), None) for war in wars}
print ('done')
wars_links = {k: v for k, v in wars_links.items() if v is not None}
print('done')
reverse_index = json.load(open(REVERSE_INDEX_FILE))
reverse_index = {k: reverse_index[str(k)] for k in wars if k is not None}
print('done')

done
done
done
done
done


In [6]:
article_users = json.load(open(os.path.join(DATA_FOLDER, '2601_article_users.json')))


In [8]:
all_users = {user for article_list in article_users.values() for user in article_list }
users_articles = {user: [] for user in all_users}
for article, users in article_users.items():
    for user in users:
        users_articles[user].append(article)

In [15]:
g = nx.Graph()
g.add_nodes_from(sorted(wars))
print(len(g.nodes()))
print(len(wars))
nx.set_node_attributes(g, name="title", values=reverse_index)

14987
14987


In [18]:
all_edges = []
start_time = time()
nodes_set = set(wars)
for i, node in enumerate(g.nodes(data=True)):
    edges = [(node[0], child) for child in wars_links[node[0]] if child in nodes_set]
    all_edges.extend(edges)
    if i % 10 == 0:
        print(i)
        print(time() - start_time)

g.add_edges_from(all_edges)

0
0.0009641647338867188
10
0.00363922119140625
20
0.006505012512207031
30
0.011832237243652344
40
0.01453542709350586
50
0.018758058547973633
60
0.020352602005004883
70
0.023852825164794922
80
0.02470850944519043
90
0.026032209396362305
100
0.026949167251586914
110
0.028321266174316406
120
0.02974677085876465
130
0.03113532066345215
140
0.03143477439880371
150
0.0337064266204834
160
0.03473949432373047
170
0.036290645599365234
180
0.04047107696533203
190
0.04908132553100586
200
0.054644107818603516
210
0.059134721755981445
220
0.06052756309509277
230
0.06154274940490723
240
0.06262016296386719
250
0.06410026550292969
260
0.06591367721557617
270
0.066436767578125
280
0.0672152042388916
290
0.06808090209960938
300
0.06922364234924316
310
0.07061195373535156
320
0.07516980171203613
330
0.07720780372619629
340
0.08646845817565918
350
0.08994507789611816
360
0.09046363830566406
370
0.09154772758483887
380
0.09428524971008301
390
0.09529399871826172
400
0.09699130058288574
410
0.097576856613

9200
0.8955404758453369
9210
0.8967115879058838
9220
0.9125027656555176
9230
0.912722110748291
9240
0.9129683971405029
9250
0.9132893085479736
9260
0.9152600765228271
9270
0.9155611991882324
9280
0.915771484375
9290
0.9174532890319824
9300
0.9177570343017578
9310
0.9180679321289062
9320
0.9184167385101318
9330
0.9188835620880127
9340
0.9192051887512207
9350
0.9195475578308105
9360
0.9197649955749512
9370
0.9201672077178955
9380
0.9204456806182861
9390
0.9206480979919434
9400
0.9208705425262451
9410
0.9237914085388184
9420
0.9240047931671143
9430
0.9243316650390625
9440
0.9246187210083008
9450
0.9247682094573975
9460
0.9250326156616211
9470
0.92523193359375
9480
0.9253346920013428
9490
0.9255774021148682
9500
0.9258487224578857
9510
0.9262833595275879
9520
0.9263894557952881
9530
0.9267497062683105
9540
0.9270024299621582
9550
0.92722487449646
9560
0.9281628131866455
9570
0.9284408092498779
9580
0.9288163185119629
9590
0.9289956092834473
9600
0.9294114112854004
9610
0.9296588897705078
9

In [19]:
g.size()

311076

In [20]:
clustering = community.best_partition(g)
print(len(set(clustering.values())))
nx.set_node_attributes(g, name='cluster', values=clustering)

85


In [21]:
title_clusters = [set() for _ in range(len(set(clustering.values())))]
node_clusters = [set() for _ in range(len(set(clustering.values())))]
for i, node in enumerate(g.nodes(data=True)):
    title = node[1]['title']
    cluster = node[1]['cluster']
    title_clusters[cluster].add(title)
    node_clusters[cluster].add(node[0])
title_clusters.sort(key=len)
node_clusters.sort(key=len)

In [22]:
h = g.subgraph([node for node in node_clusters[-1]])

h.size()

smaller_clustering = community.best_partition(h)
print(len(set(smaller_clustering.values())))

smaller_title_clusters = [set() for _ in range(len(set(smaller_clustering.values())))]
smaller_node_clusters = [set() for _ in range(len(set(smaller_clustering.values())))]
for i, node in enumerate(h.nodes(data=True)):
    title = node[1]['title']
    cluster = smaller_clustering[node[0]]
    smaller_title_clusters[cluster].add(title)
    smaller_node_clusters[cluster].add(node[0])
smaller_title_clusters.sort(key=len)
smaller_node_clusters.sort(key=len)

6


In [23]:
smaller_title_clusters[-1]
len(smaller_title_clusters[-1])

1231

In [24]:
i = g.subgraph([node for node in smaller_node_clusters[-1]])

i.size()

even_smaller_clustering = community.best_partition(i)
print(len(set(even_smaller_clustering.values())))

even_smaller_title_clusters = [set() for _ in range(len(set(even_smaller_clustering.values())))]
even_smaller_node_clusters = [set() for _ in range(len(set(even_smaller_clustering.values())))]
for i, node in enumerate(i.nodes(data=True)):
    title = node[1]['title']
    cluster = even_smaller_clustering[node[0]]
    even_smaller_title_clusters[cluster].add(title)
    even_smaller_node_clusters[cluster].add(node[0])
even_smaller_title_clusters.sort(key=len)
even_smaller_node_clusters.sort(key=len)

8


In [32]:
even_smaller_title_clusters[-10]

IndexError: list index out of range

In [33]:
final_node_clusters = node_clusters[:-1] + smaller_node_clusters[:-1] + even_smaller_node_clusters
final_title_clusters = title_clusters[:-1] + smaller_title_clusters[:-1] + even_smaller_title_clusters
final_node_clusters.sort(key=len, reverse=True)
final_title_clusters.sort(key=len, reverse=True)

In [57]:
title_clusters[-4]

{'Auld Lang Syne',
 'Century break',
 'Gough Whitlam',
 'Warnborough College',
 'First Fleet',
 'McDonnell Douglas MD-12',
 'Spring Hill Fair',
 'City status in the United Kingdom',
 'Middlesex',
 'UEFA Euro 2012',
 'Conservative Party (UK)',
 '2018 Commonwealth Games',
 'Drake Circus Shopping Centre',
 'Bradford',
 'M4 motorway',
 'Lauren Booth',
 'Neymar',
 'Pet door',
 'Never Forget (musical)',
 'Harry Potter (character)',
 'Countries of the United Kingdom',
 'British Armed Forces',
 'Punched card',
 'Lady Jane Grey',
 'Quebec referendum, 1995',
 'Concorde',
 'Patrick Pearse',
 'Laudabiliter',
 'Geoffrey Edelsten',
 'Victoria Cross (Canada)',
 'Dorset',
 'Volunteer (Irish republican)',
 'Disappearance of Madeleine McCann',
 'Albus Dumbledore',
 'Royal Canadian Mounted Police',
 'Kitchener, Ontario',
 'Stephen Fry',
 'St Pancras railway station',
 'Mexico national football team',
 'Dalton McGuinty',
 '2011 UK Open',
 '2011 Novak Djokovic tennis season',
 'Pavlova (food)',
 'The Time 

In [23]:
final_clustering = {}
for i, node_cluster in enumerate(final_node_clusters):
    for node in node_cluster:
        final_clustering[node] = i
nx.set_node_attributes(g, name='cluster', values=final_clustering)

In [24]:
nx.write_gpickle(g, "final_graph.pickle")

Dump of the graph visualization

In [25]:
"""graph_dict = {"graph": [],
             "links": [],
             "nodes": [],
             "directed": False,
             "multigraph": False}

def cluster_score(cluster_number):
    total_number = len(final_node_clusters)
    return ((cluster_number * 107) % total_number) /float(total_number)

node_index = {node: i for i, node in enumerate(g.nodes())}
indexed_edges = [{"source": node_index[edge[0]], "target": node_index[edge[1]]} for edge in g.edges]
indexed_nodes = [{"size": node[1]["controversy_value"], "score": cluster_score(node[1]["cluster"]), "type": "circle", "id": node[1]["title"]} for node in g.nodes(data=True)]

graph_dict["links"] = indexed_edges
graph_dict["nodes"] = indexed_nodes

json.dump(graph_dict, open("d3graph.json", "w"), indent=2)
print(len(final_node_clusters))

final_title_clusters[450]"""